In [ ]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, GRPO_WRITING_CONFIG, SMOL_LM_135M

cfg = GRPO_WRITING_CONFIG
cfg.train_batch_size = 2
cfg.num_generations = 2
cfg.model_id_or_path = SMOL_LM_135M
cfg.max_samples = 10

In [ ]:
wrapper = TrainerWrapper(cfg, True)

In [ ]:
wrapper.init_model()

In [ ]:
wrapper.init_data_module()

In [ ]:
wrapper.data_module.train_dataset

In [ ]:
wrapper.init_trainer()

In [ ]:
wrapper.train()

In [9]:
import pandas as pd
dataset_pd = pd.read_parquet(
    "../dataset_files/gutenberg_backtranslate_from_txt.parquet"
)

In [ ]:
completion, instruction = dataset_pd.loc[500]['text'], dataset_pd.loc[500]['instruction']
wrapper.data_module.reward_functions()[0]([instruction], [completion])